---
## pandas
En el apartado de pandas queremos dar a conocer algunas de las funciones "ocultas" de pandas y que pueden ser muy útiles a la hora de manipular un dataset. En concreto, buscamos:

1. Entender en que consiste el **pandas groupby** y como podemos utilizarlo para hacer cálculo de manera **rápida, limpia y eficiente.**
2. Aprender a realizar **varios cálculos a la vez sobre diferentes columnas (CORE IDEA)** de un df todo dento del pandas groupby.
3. Entender el método de **pandas transform (CORE IDEA)**.
4. Definir y utilizar **funciones propias (custom functions)** dentro de un groupby.
5. Ver como podemos utilizar las **funciones propias** para hacer un cálculo dentro de un groupby utilizando **A LA VEZ** la información de 2 columnas diferentes (o más).
6. Aprender filtros avanzados de la librerías pandas.
---

# Imports del notebook
<a id='imports_notebook'></a>
[Volver al índice](#index)

En este apartado hacemos los principales imports del notebook.<br>
Sobre todo vamos a trabajar con **numpy**, **pandas** y **sklearn**.

In [1]:
import time
import os
import warnings

import numpy as np
import pandas as pd

In [2]:
CWD = os.getcwd()

In [3]:
os.path.join(CWD, "data")

'c:\\Mariano\\1. Master en Data Science\\8. GIT\\portfolio\\data'

In [4]:
DATA_PATH = os.path.join(CWD, "data")

print(CWD)
print(DATA_PATH)

c:\Mariano\1. Master en Data Science\8. GIT\portfolio
c:\Mariano\1. Master en Data Science\8. GIT\portfolio\data


In [5]:
print("Working with these versions of libraries\n")
print(f"Numpy version {np.__version__}")
print(f"Pandas version {pd.__version__}")

Working with these versions of libraries

Numpy version 1.23.5
Pandas version 1.5.3


Para todos los ejercicios de **pandas**, vamos a utilizar un dataset dummy.

In [6]:
def get_category(nr_rows):
    '''
    Generates random category.
    '''
    categories = ["A", "B", "C", "X", "Y", "Z"]
    random_categories = np.random.choice(categories, size = nr_rows)
    
    return random_categories

In [7]:
def generate_dummy_dataframe(nr_rows, nr_columns, nr_categoricals):
    '''
    Generates a dummy DataFrame.
    '''
    la = []
    
    for nr_ in range(nr_columns):
        la.append(np.random.randint(-1000, 1000, size = nr_rows))
            
    df = pd.DataFrame(la).T
    
    cat_index = [cat + str(i) for i, cat in enumerate(get_category(nr_rows = nr_rows))]
    df.index = cat_index
    
    dummy_column_names = ["Column_{}".format(i) for i in range(1, nr_columns + 1)]
    
    df.columns = dummy_column_names
    
    for nr_categorical in range(nr_categoricals):
        df["Categorical_{}".format(nr_categorical + 1)] = get_category(nr_rows = nr_rows)
    
    return df
    

In [24]:
# df = generate_dummy_dataframe(nr_rows = 1_000, nr_columns = 3, nr_categoricals = 3)
# df.to_csv(os.path.join(DATA_PATH, "pd_sklearn_dummy.csv"), index = False)

In [8]:
df = pd.read_csv(os.path.join(DATA_PATH, "pd_sklearn_dummy.csv"))

Veamos nuestro dataframe

In [9]:
df.head(5)

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
0,-792,-125,51,X,Y,Z
1,-866,-397,723,Y,B,A
2,192,513,942,X,C,Y
3,73,984,-620,A,B,A
4,-225,168,-50,A,B,C


## ¿Que es el pandas groupby?

Veamos un ejemplo práctico con nuestro dataset.

### Pregunta 1: hacer la suma de Column_1 cuando Categorical_1 == "B"

In [10]:
sorted(df["Categorical_1"].unique())

['A', 'B', 'C', 'X', 'Y', 'Z']

In [11]:
df[df["Categorical_1"] == "B"]["Column_1"].sum()

1006

In [12]:
# posible solución
df[df["Categorical_1"] == "B"]["Column_1"].sum()

1006

Cuando queremos hacer un cálculo rápido y sencillo, podemos utilizar los **filtros de pandas** para hacer una selección de un subconjunto de datos y aplicar nuestra fórmula.

No obstante, ¿Que pasaría si necesitamos calcular esta suma para todas las categorías que hay en Categorical_1?

Una implementación rápida sería utilizar una for loop, como en el siguiente ejemplo.

In [13]:
RESULT_DICT = {}

for cat_ in sorted(df["Categorical_1"].unique()):
    df_ = df[df["Categorical_1"] == cat_]
    suma_ = df_["Column_1"].sum()
    
    RESULT_DICT[cat_] = suma_

In [14]:
RESULT_DICT

{'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769}

In [15]:
df.groupby("Categorical_1")["Column_1"].sum().to_dict()

{'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769}

In [16]:
gb_ = df.groupby("Categorical_1")

In [17]:
gb_["Column_1"]

In [18]:
df[df["Categorical_1"] == "B"].head(3)

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
8,989,527,-988,B,A,Z
10,-349,-306,390,B,A,B
17,-801,-787,306,B,X,Y


In [19]:
gb_.get_group(("B"))["Column_1"]

8      989
10    -349
17    -801
18     169
23    -771
      ... 
966     90
976   -394
978    619
983    196
995   -750
Name: Column_1, Length: 174, dtype: int64

In [20]:
# posible solución
result_dict = {}

for category in sorted(df["Categorical_1"].unique()):
    result_dict[category] = df[df["Categorical_1"] == category]["Column_1"].sum()
    
result_dict

{'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769}

La implementación anterior es válida, pero hay un forma mucho mejor de hacerlo. Utilizar el pandas groupby.

In [21]:
# creamos un objeto de pandas groupby y lo asignamos a gb_df
gb_df = df.groupby("Categorical_1")
gb_df

In [22]:
# Hice el groupby por Categorical_1, la categoría A existe dentro de Categorical_1
# Por este motivo, cuando hago get_group(("A")) veo que todos los elementos de Categorical_1 == "A"
gb_df.get_group(("A")).head()

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
3,73,984,-620,A,B,A
4,-225,168,-50,A,B,C
7,707,786,-570,A,A,Z
9,509,-522,795,A,B,A
14,-495,-227,-548,A,Z,Z


In [23]:
# lo mismo pero con el grupo C
gb_df.get_group(("C")).head()

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
5,59,990,-185,C,X,C
11,863,-718,736,C,X,Z
15,543,-164,-787,C,A,A
28,-32,103,-729,C,X,C
29,-825,-828,-543,C,Y,A


In [24]:
# que pasaría si intento buscar el grupo I
try:
    gb_df.get_group(("I")).head()
except:
    print('El gb_df.get_group(("I")).head() ha fallado! :(')

El gb_df.get_group(("I")).head() ha fallado! :(


¡La ejecución anterior nos ha dado un error!

### Podemos pensar en un groupby como un conjunto de dataframes pequeños.<br>
De hecho, hacer el get_group es idéntico a hacer un filtro de pandas groupby.

In [25]:
df[df["Categorical_1"] == "A"].head()

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
3,73,984,-620,A,B,A
4,-225,168,-50,A,B,C
7,707,786,-570,A,A,Z
9,509,-522,795,A,B,A
14,-495,-227,-548,A,Z,Z


In [26]:
gb_df.get_group(("A")).head()

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
3,73,984,-620,A,B,A
4,-225,168,-50,A,B,C
7,707,786,-570,A,A,Z
9,509,-522,795,A,B,A
14,-495,-227,-548,A,Z,Z


In [27]:
df[df["Categorical_1"] == "A"].equals(gb_df.get_group(("A"))) # comprobamos que son iguales

True

<a id='tip_1_pandas_gb_index'></a>
### Si nos fijamos de manera detenida, vemos que el index del df filtrado, es el mismo que el index del get_group.

Esta parte va a ser muy importante cuando vamos a trabajar con **pandas transform**.

De momento, nos quedamos con la idea de que hacer un filtro o get_group de groupby **me permite acceder a un subconjunto del dataframe en cuestión.**

In [28]:
print(result_dict)

{'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769}


In [29]:
# best praxis
gb_df["Column_1"].sum().to_dict()

{'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769}

### Pandas transform (CORE IDEA)
<a id='pandas_transform'></a>

### Pregunta 2: añadir una nueva columna al df que sea la suma total de Column_1 respetando cada categoría de Categorical_1

In [30]:
RESULT_DICT

{'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769}

In [31]:
df.head(3)

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3
0,-792,-125,51,X,Y,Z
1,-866,-397,723,Y,B,A
2,192,513,942,X,C,Y


In [32]:
suma_ = df.groupby('Categorical_1')['Column_1'].sum()

In [33]:
suma_dict = suma_.to_dict()

In [34]:
suma_dict

{'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769}

In [35]:
df["Solucion_1"] = df["Categorical_1"].map(suma_dict)

In [36]:
df

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3,Solucion_1
0,-792,-125,51,X,Y,Z,11813
1,-866,-397,723,Y,B,A,6122
2,192,513,942,X,C,Y,11813
3,73,984,-620,A,B,A,1988
4,-225,168,-50,A,B,C,1988
...,...,...,...,...,...,...,...
995,-750,397,-833,B,C,Y,1006
996,-508,917,-443,Y,A,C,6122
997,213,-781,692,X,B,C,11813
998,628,-926,-316,X,Z,X,11813


In [37]:
df.groupby("Categorical_1")["Column_1"].sum()

Categorical_1
A     1988
B     1006
C    -1595
X    11813
Y     6122
Z     9769
Name: Column_1, dtype: int64

In [38]:
df.groupby("Categorical_1")["Column_1"].transform("sum")

0      11813
1       6122
2      11813
3       1988
4       1988
       ...  
995     1006
996     6122
997    11813
998    11813
999    -1595
Name: Column_1, Length: 1000, dtype: int64

In [39]:
np.sum

<function numpy.sum(a, axis=None, dtype=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)>

In [40]:
np.sum([1, 2])

3

In [41]:
df["Sol_2"] = df.groupby("Categorical_1")["Column_1"].transform(
    np.sum
)


In [42]:
df

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3,Solucion_1,Sol_2
0,-792,-125,51,X,Y,Z,11813,11813
1,-866,-397,723,Y,B,A,6122,6122
2,192,513,942,X,C,Y,11813,11813
3,73,984,-620,A,B,A,1988,1988
4,-225,168,-50,A,B,C,1988,1988
...,...,...,...,...,...,...,...,...
995,-750,397,-833,B,C,Y,1006,1006
996,-508,917,-443,Y,A,C,6122,6122
997,213,-781,692,X,B,C,11813,11813
998,628,-926,-316,X,Z,X,11813,11813


In [43]:
# posible solución
result_dict_gb = df.groupby("Categorical_1")["Column_1"].sum().to_dict()

df["New_col_1"] = df["Categorical_1"].map(result_dict_gb)

df

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3,Solucion_1,Sol_2,New_col_1
0,-792,-125,51,X,Y,Z,11813,11813,11813
1,-866,-397,723,Y,B,A,6122,6122,6122
2,192,513,942,X,C,Y,11813,11813,11813
3,73,984,-620,A,B,A,1988,1988,1988
4,-225,168,-50,A,B,C,1988,1988,1988
...,...,...,...,...,...,...,...,...,...
995,-750,397,-833,B,C,Y,1006,1006,1006
996,-508,917,-443,Y,A,C,6122,6122,6122
997,213,-781,692,X,B,C,11813,11813,11813
998,628,-926,-316,X,Z,X,11813,11813,11813


La implementación anterior utiliza una variable temporal, en nuestro caso result_dict_gb para guardar los resultados y después mappear el resultado al df.  Puede parecer una buena forma de hacerlo hasta que conocemos al pandas **transform**.

In [44]:
# best praxis
df["New_col_1_bp"] = df.groupby("Categorical_1")["Column_1"].transform(np.sum)

Pandas **transform** es una mejor solución porque permite conseguir el mismo resultado
en 1 única línea de código. No hay necesidad de guardar nada de manera temporal. Con esto conseguimos un código más limpio y elegante.

Internamente dentro de groupby lo que ocurre es:
1. Calculamos la suma de Column_1 para cada categoría.
2. Hacemos el merge con el df orignal, por el index del dataframe.

Si nos fijamos en la siguiente línea de código, el index es idéntico al df original.

In [45]:
df.groupby("Categorical_1")["Column_1"].transform(np.sum).head()

0    11813
1     6122
2    11813
3     1988
4     1988
Name: Column_1, dtype: int64

In [46]:
df.head(5)

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3,Solucion_1,Sol_2,New_col_1,New_col_1_bp
0,-792,-125,51,X,Y,Z,11813,11813,11813,11813
1,-866,-397,723,Y,B,A,6122,6122,6122,6122
2,192,513,942,X,C,Y,11813,11813,11813,11813
3,73,984,-620,A,B,A,1988,1988,1988,1988
4,-225,168,-50,A,B,C,1988,1988,1988,1988


### Groupby eficiente
<a id='pandas_gbe'></a>

En más de una ocasión, queremos hacer varios cálculos dentro de un pandas groupby

---
Habitualmente el patrón que se sigue es:

1. variable_1 = df.groupby(["CAT_1", "CAT_2"])["COLUMNA_1"].funcion_agregación()
2. variable_2 = df.groupby(["CAT_1", "CAT_2"])["COLUMNA_2"].otra_funcion_agregación()
3. df_resumen = pd.concat([variable_1, variable_2], axis = 1)

### Pregunta 3: crear un df_resumen que contenga la suma, media (para Column_1) y un contador de los valores positivos para la columna Column_2, agrupados por Categorical_1

El resultado que nos debería dar un df similar a este.

In [47]:
result_data_dict = {
    'Sum_column_1': {'A': 1988, 'B': 1006, 'C': -1595, 'X': 11813, 'Y': 6122, 'Z': 9769},
    'Mean_column_1': {'A': 10.57, 'B': 5.78, 'C': -9.27, 'X': 78.75, 'Y': 35.59, 'Z': 67.84},
    'Counter_non_negative_values_column_2': {'A': 101, 'B': 85, 'C': 89, 'X': 77, 'Y': 91,'Z': 73}
}

In [48]:
pd.DataFrame(data = result_data_dict)

,Sum_column_1,Mean_column_1,Counter_non_negative_values_column_2
A,1988,10.57,101
B,1006,5.78,85
C,-1595,-9.27,89
X,11813,78.75,77
Y,6122,35.59,91
Z,9769,67.84,73


In [49]:
suma_ = df.groupby("Categorical_1")["Column_1"].sum()
mean_ = df.groupby("Categorical_1")["Column_1"].mean()
counter_ = df[df["Column_2"] >= 0].groupby("Categorical_1").size()

dfgb = pd.concat([suma_, mean_, counter_], axis = 1)

dfgb.columns = ["Suma_Column_1", "Mean_Column_1", "Counter_NN"]

dfgb

,Suma_Column_1,Mean_Column_1,Counter_NN
Categorical_1,,,
A,1988,10.574468,101
B,1006,5.781609,85
C,-1595,-9.273256,89
X,11813,78.753333,77
Y,6122,35.593023,91
Z,9769,67.840278,73


In [50]:
series_ = df[df["Categorical_1"] == "B"]["Column_1"]

In [51]:
def counter_positive_values(series):
    positive_values = len(series[series >= 0])
    return f"We have a total of {positive_values} positive values"

In [52]:
counter_positive_values(series = series_)

'We have a total of 89 positive values'

In [53]:
counter_positive_values(
    series = df.groupby("Categorical_1").get_group("C")["Column_2"]
)

'We have a total of 89 positive values'

In [54]:
resumen = df.groupby("Categorical_1").agg(
    Suma_Column_1_gb = ("Column_1", np.sum),
    Mean_Column_1_gb = ("Column_1", np.mean),
    Counter_Column_2_gb = ("Column_2", counter_positive_values)
)

In [55]:
resumen

,Suma_Column_1_gb,Mean_Column_1_gb,Counter_Column_2_gb
Categorical_1,,,
A,1988,10.574468,We have a total of 101 positive values
B,1006,5.781609,We have a total of 85 positive values
C,-1595,-9.273256,We have a total of 89 positive values
X,11813,78.753333,We have a total of 77 positive values
Y,6122,35.593023,We have a total of 91 positive values
Z,9769,67.840278,We have a total of 73 positive values


In [56]:
# posible solución
suma_column_1_categorical_1 = df.groupby("Categorical_1")["Column_1"].sum()
mean_column_1_categorical_1 = df.groupby("Categorical_1")["Column_1"].mean()
positive_counter = df.groupby("Categorical_1")["Column_2"].agg(lambda series: len(series[series >= 0]))

df_resumen = pd.concat([
    suma_column_1_categorical_1,
    mean_column_1_categorical_1,
    positive_counter
], axis = 1)

df_resumen.columns = ["Sum_column_1", "Mean_column_1", "Counter_non_negative_values_column_2"]

del suma_column_1_categorical_1, mean_column_1_categorical_1, positive_counter

df_resumen.round(2)

,Sum_column_1,Mean_column_1,Counter_non_negative_values_column_2
Categorical_1,,,
A,1988,10.57,101
B,1006,5.78,85
C,-1595,-9.27,89
X,11813,78.75,77
Y,6122,35.59,91
Z,9769,67.84,73


Si miramos de manera crítica la celda anterior, podemos ver las siguientes ineficiencias:
1. Generamos 3 variables temporales que no nos hacen falta a posteriori.
2. Debemos añadir varias líneas de código para concatenar estos resultados.
3. Tenemos que hacer limpieza de las variables temporales (buena praxis).
4. Debemos renombrar las columnas del df_resumen.

Todos los pasos anteriores para conseguir un df_resumen.

Veamos una forma mucho más elegante y eficiente de hacerlo.

In [57]:
# best praxis
df.groupby("Categorical_1").agg(
    Sum_column_1 = ("Column_1", np.sum),
    Max_column_1 = ("Column_1", np.max),
    Mean_column_1 = ("Column_1", np.mean),
    Counter_non_negative_values_column_2 = ("Column_2", lambda series: len(series[series >= 0])),
    Sum_non_negative_values_column_2 = ("Column_2", lambda series: np.sum(series[series >= 0])),
).round(2)

,Sum_column_1,Max_column_1,Mean_column_1,Counter_non_negative_values_column_2,Sum_non_negative_values_column_2
Categorical_1,,,,,
A,1988,987,10.57,101,51758
B,1006,989,5.78,85,33793
C,-1595,958,-9.27,89,43271
X,11813,995,78.75,77,35980
Y,6122,999,35.59,91,48768
Z,9769,998,67.84,73,33750


Conseguimos el mismo resultado que antes, pero de una manera más rápida y elegante. 

**Debemos fijarnos en que en realidad, hacemos 2 operaciones adicionales y todo con menos líneas de código.**

### Funciones personalizadas
<a id='pandas_custom_func'></a>

En el ejemplo anterior, para calcular **Counter_non_negative_values_column_2** tuvimos que crear una función personalizada (custom function) en forma de ***lambda***.

Esto es de extrema relevancia porque significa que dentro de un **pandas groupby** yo puedo utilizar no solo las funciones tipo: suma, media, máximo etc. sino **cualquier función siempre y cuando haga un cálculo y devuelva un resultado agregado.** De hecho, el resultado podría hasta ser un string. 

Veamos algunos ejemplos.

In [58]:
def custom_func_non_negative_sum(series):
    '''
    Sums non negative values from a pandas series
    '''
    return np.sum(series[series >= 0])

La función anterior es idéntica a la siguiente función anónima **lambda series: np.sum(series[series >= 0])**

In [59]:
df.groupby("Categorical_1")["Column_2"].agg(custom_func_non_negative_sum)

Categorical_1
A    51758
B    33793
C    43271
X    35980
Y    48768
Z    33750
Name: Column_2, dtype: int64

Por supuesto la función anterior la podemos utilizar con un pandas transform

In [60]:
df["New_col_2_cf_bp"] =\
df.groupby("Categorical_1")["Column_2"].transform(
    custom_func_non_negative_sum
)

In [61]:
df.head(1)

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3,Solucion_1,Sol_2,New_col_1,New_col_1_bp,New_col_2_cf_bp
0,-792,-125,51,X,Y,Z,11813,11813,11813,11813,35980


Antes hemos comentado, que la función de agregación customizada puede hasta devolver un string como resultado.

Veamos un ejemplo rápido.

In [62]:
def custom_func_returns_string(series):
    '''
    Custom function that returns a string as a result
    '''
    positive_values = len(series[series >= 0])
    negative_values = len(series[series < 0])
    
    return f"Positive Values: {positive_values} Negative Values: {negative_values}"

In [63]:
print(df.groupby("Categorical_1")["Column_1"].agg(custom_func_returns_string))

Categorical_1
A    Positive Values: 99 Negative Values: 89
B    Positive Values: 89 Negative Values: 85
C    Positive Values: 91 Negative Values: 81
X    Positive Values: 82 Negative Values: 68
Y    Positive Values: 83 Negative Values: 89
Z    Positive Values: 81 Negative Values: 63
Name: Column_1, dtype: object


In [64]:
df["New_col_2_cfs_bp"] = df.groupby("Categorical_1")["Column_1"].transform(custom_func_returns_string)

In [65]:
df.sample(5)

,Column_1,Column_2,Column_3,Categorical_1,Categorical_2,Categorical_3,Solucion_1,Sol_2,New_col_1,New_col_1_bp,New_col_2_cf_bp,New_col_2_cfs_bp
469,538,474,116,B,X,B,1006,1006,1006,1006,33793,Positive Values: 89 Negative Values: 85
127,-188,398,-320,C,C,Y,-1595,-1595,-1595,-1595,43271,Positive Values: 91 Negative Values: 81
793,-797,-697,-856,X,X,A,11813,11813,11813,11813,35980,Positive Values: 82 Negative Values: 68
380,-846,205,942,C,B,C,-1595,-1595,-1595,-1595,43271,Positive Values: 91 Negative Values: 81
220,-962,-777,-974,A,A,C,1988,1988,1988,1988,51758,Positive Values: 99 Negative Values: 89


En la práctica no obstante, usamos este tipo de funciones sólo cuando queremos hacer un reporting sobre un dataframe e incluirlo en un informe final.

### Nueva columna utilizando la información de 2 columnas
<a id='pandas_custom_func_2_cols'></a>

Si hacemos un breve recap de lo visto hasta ahora, sabemos que:
1. Dentro de un groupby hay un montón de dfs "pequeños". Estos dataframes no se solapan con otros porque tienen categorías únicas (en función de las columnas por las cuales se ha hecho el groupby).
2. De alguna manera, hacer un groupby y llamar get_group (que es lo que ocurre dentro de un groupby pero para todas las categorías) es similar a hacer un filtro de df.
3. Al hacer un filtro o groupby, sigo operando todo el rato con un df. Si sigo operando con un df, puedo acceder a cualquier columna del df.
4. Por tanto, puedo realizar una función de agregación teniendo en cuenta la información de más de una columna, sin necesidad de **iterar** sobre el df.

### Pregunta 4: calcular la media de la Column_1 cuando tengo en la Column_2 más valores positivos que negativos (len_positivos >= len_negativos), en caso contrario devolver -1 todo esto dentro de un groupby.

In [66]:
RESULT_DICT = {}

for cat_ in sorted(df["Categorical_1"].unique()):
    
    df_ = df[df["Categorical_1"] == cat_]
    
    positive_values = (df_["Column_2"] >= 0).sum()
    negative_values = (df_["Column_2"] < 0).sum()

    if positive_values >= negative_values:
        result_ = df_["Column_1"].mean()
    else:
        result_ = -1
        
    RESULT_DICT[cat_] = result_
    
RESULT_DICT

{'A': 10.574468085106384,
 'B': -1,
 'C': -9.273255813953488,
 'X': 78.75333333333333,
 'Y': 35.593023255813954,
 'Z': 67.84027777777777}

In [67]:
def custom_agg_2_columns_gb(df):
    
    positive_values = (df["Column_2"] >= 0).sum()
    negative_values = (df["Column_2"] < 0).sum()
    
    if positive_values >= negative_values:
        result_ = df["Column_1"].mean()
        return result_
    else:
        result_ = -1
        return result_

In [68]:
df.groupby("Categorical_1").apply(custom_agg_2_columns_gb)

Categorical_1
A    10.574468
B    -1.000000
C    -9.273256
X    78.753333
Y    35.593023
Z    67.840278
dtype: float64

In [69]:
# posible solución
d_ = {}

for category in sorted(df["Categorical_1"].unique()):

    sample_df = df[
        (df["Categorical_1"] == category)
    ]

    value_counts = (sample_df["Column_2"] >= 0).value_counts()
    
    true_values = value_counts.loc[True]
    false_values = value_counts.loc[False]
    
    if true_values >= false_values:
        d_[category] = sample_df["Column_1"].mean()
    else:
        d_[category] = -1

d_

{'A': 10.574468085106384,
 'B': -1,
 'C': -9.273255813953488,
 'X': 78.75333333333333,
 'Y': 35.593023255813954,
 'Z': 67.84027777777777}

In [70]:
# best praxis
def custom_agg_function_2_cols(df):
    '''
    Custom function that uses values from 1 column to calculate the value for another column.
    '''
    value_counts = (df["Column_2"] >= 0).value_counts()
    true_values = value_counts.loc[True]
    false_values = value_counts.loc[False]
    
    if true_values >= false_values:
        return df["Column_1"].mean()
    else:
        return -1

df.groupby("Categorical_1").apply(custom_agg_function_2_cols)

Categorical_1
A    10.574468
B    -1.000000
C    -9.273256
X    78.753333
Y    35.593023
Z    67.840278
dtype: float64